Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

SCRAPING DATA FROM WIKIPEDIA

In [1]:
!pip install beautifulsoup4

     |████████████████████████████████| 112kB 5.3MB/s eta 0:00:01


In [2]:
%%capture
! pip install seaborn

In [3]:
import seaborn as sns
import bs4 as pad# library for scraping
from bs4 import BeautifulSoup as soup

In [4]:
import requests  # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.2 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.21.0-py_0

The following packages will be UPDATED:

  openssl                                 1.1.1f-h516909a_0 --> 1.1.1g-h516909a_0



openssl-1.1.1g       | 2.1 MB    | ############

In [5]:
!conda install lxml --yes

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.1.1   |                0         125 KB
    certifi-2020.4.5.1         |           py36_0         155 KB
    libxslt-1.1.33             |       h7d1a2b0_0         426 KB
    lxml-4.5.0                 |   py36hefd8a0e_0         1.4 MB
    openssl-1.1.1g             |       h7b6447c_0         2.5 MB
    ------------------------------------------------------------
                                           Total:         4.6 MB

The following NEW packages will be INSTALLED:

  libxslt            pkgs/main/linux-64::libxslt-1.1.33-h7d1a2b0_0
  lxml               pkgs/main/linux-64::lxml-4.5.0-py36hefd8a0e_0

The following packages will be SUPERSEDED by a higher-priority channel:



In [11]:
my_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [12]:
# Download the HTMl code for the Canada Dta
from urllib.request import urlopen as uReq
uClient= uReq(my_url)
page_html=uClient.read()
uClient.close()



In [13]:
# Parse the data and show the structure
page_soup = soup(page_html, 'html.parser')
[type(item) for item in list(page_soup.children)] 


[bs4.element.NavigableString,
 bs4.element.Doctype,
 bs4.element.NavigableString,
 bs4.element.Tag,
 bs4.element.NavigableString]

In [20]:
# Select the table and pass the data to Panda Dataframe
table=page_soup.find_all('table')[0]
df=pd.read_html(str(table))
neighborhoods=df[0]
neighborhoods.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [21]:
# Propertiies of the dataframe
neighborhoods.size

540

In [22]:
# Ignoring the rows with Not assigned Boroughs

neighborhoods= neighborhoods[neighborhoods.Borough!='Not assigned']
neighborhoods.size

309

We checked that There is not a set of Neighborhoods with Not assigned Neighborhood, so we don't need to change any value. 

In [23]:
# rows with Neighborhood with Not assigned Neighborhood
NonNeighborhood= neighborhoods[neighborhoods.Neighborhood=='Not assigned']
NonNeighborhood.size

0

In [81]:
# Now we chage the '/' by ',' in the Neighbohood

# neighborhoods['Neighborhood']=neighborhoods['Neighborhood'].apply(lambda x: x.replace('/', ',')) // This a possibility

neighborhoods['Neighborhood'] = neighborhoods['Neighborhood'].str.replace('/',',')

neighborhoods.sort_values(by=['Postal code'], inplace=True)
neighborhoods


,Postal code,Borough,Neighborhood
9,M1B,Scarborough,"Malvern , Rouge"
18,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
27,M1E,Scarborough,"Guildwood , Morningside , West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
107,M9P,Etobicoke,Westmount
116,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov..."
143,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam..."


In [31]:
# Final Data step 1:

neighborhoods.shape

(103, 3)

In [40]:
!conda install -c conda-forge geocoder  --y

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    click-7.1.1                |     pyh8c360ce_0          64 KB  conda-forge
    future-0.18.2              |   py36h9f0ad1d_1         714 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         837 KB

The following NEW packages will be INSTALLED:

  click              conda-forge/noarch::click-7.1.1-pyh8c360ce_0
  decorator          conda-forge/noarch::decorator-4.4.2-py_0
  future             conda-forge/linux-64::future-0.18.2-py36h9f0ad1d_1
  geocoder       

In [75]:
#using the file Link we can get the Latitudes and Longitud, We have 103 rows

path='http://cocl.us/Geospatial_data'
location=pd.read_csv(path)
location.shape


(103, 3)

In [76]:
# We sort the data to add to the Dataframe of neighborhoods which is sorted too
location.sort_values(by=['Postal Code'])
location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [82]:
# We add The Latitudes

neighborhoods['Latitude'] = location['Latitude'].values
neighborhoods

,Postal code,Borough,Neighborhood,Latitude
9,M1B,Scarborough,"Malvern , Rouge",43.806686
18,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535
27,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573
36,M1G,Scarborough,Woburn,43.770992
45,M1H,Scarborough,Cedarbrae,43.773136
...,...,...,...,...
98,M9N,York,Weston,43.706876
107,M9P,Etobicoke,Westmount,43.696319
116,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.688905
143,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416


In [83]:
# We add The Longitudes
neighborhoods['Longitude'] = location['Longitude'].values
neighborhoods

,Postal code,Borough,Neighborhood,Latitude,Longitude
9,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
18,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
27,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
36,M1G,Scarborough,Woburn,43.770992,-79.216917
45,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
107,M9P,Etobicoke,Westmount,43.696319,-79.532242
116,M9R,Etobicoke,"Kingsview Village , St. Phillips , Martin Grov...",43.688905,-79.554724
143,M9V,Etobicoke,"South Steeles , Silverstone , Humbergate , Jam...",43.739416,-79.588437
